UK trade in services: all countries, non-seasonally adjusted

In [1]:
from gssutils import *

if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path

    session = CacheControl(requests.Session(),
                           cache=FileCache('.cache'),
                           heuristic=LastModified())

    sourceFolder = Path('in')
    sourceFolder.mkdir(exist_ok=True)

    inputURL = 'https://www.ons.gov.uk/file?uri=/businessindustryandtrade/internationaltrade/datasets/uktradeinservicesallcountriesnonseasonallyadjusted/current/tradeinservicesbycountry.xlsx'
    inputFile = sourceFolder / 'tradeinservicesbycountry.xlsx'
    response = session.get(inputURL)
    with open(inputFile, 'wb') as f:
      f.write(response.content)    

In [2]:
tab = loadxlstabs(inputFile, sheetids='TiS by country')[0]

Loading in\tradeinservicesbycountry.xlsx which has size 50730 bytes
Table names: ['TiS by country']


In [3]:
observations = tab.excel_ref('C7').expand(DOWN).expand(RIGHT).is_not_blank()

In [4]:
Year = tab.excel_ref('C4').expand(RIGHT).is_not_whitespace()

In [5]:
Flow = tab.excel_ref('B').expand(DOWN).by_index([5,253])

In [6]:
geo1 = tab.excel_ref('A7').expand(DOWN).is_not_blank()

In [7]:
Dimensions = [
            HDim(Year,'Period',DIRECTLY,ABOVE),
            HDim(geo1,'ONS Partner Geography',DIRECTLY,LEFT),
            HDim(Flow, 'Flow',CLOSEST,ABOVE),
            HDimConst('Measure Type', 'GBP Total'),
            HDimConst('Unit','gbp-million')
            ]

In [8]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [9]:
new_table = c1.topandas()

In [10]:
new_table['Period'] = 'quarter/' + new_table['Period'].astype(str).str[0:4]+ '-' +   new_table['Period'].astype(str).str[-2:]              

In [11]:
new_table.head()

,OBS,Period,ONS Partner Geography,Flow,Measure Type,Unit
0,228.0,quarter/2016-Q1,AT,Exports,GBP Total,gbp-million
1,254.0,quarter/2016-Q2,AT,Exports,GBP Total,gbp-million
2,278.0,quarter/2016-Q3,AT,Exports,GBP Total,gbp-million
3,275.0,quarter/2016-Q4,AT,Exports,GBP Total,gbp-million
4,292.0,quarter/2017-Q1,AT,Exports,GBP Total,gbp-million


In [12]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [13]:
new_table['Flow'] = new_table['Flow'].str.lower()

In [14]:
new_table['Seasonal Adjustment'] =  'NSA'

In [15]:
new_table.dtypes

Value                    float64
Period                    object
ONS Partner Geography     object
Flow                      object
Measure Type              object
Unit                      object
Seasonal Adjustment       object
dtype: object

In [16]:
# new_table['Value'] = pd.to_numeric(new_table['Value'], errors='coerce').fillna(0)

In [17]:
new_table['Value'] = new_table['Value'].astype(int)

In [18]:
new_table['Pink Book Services'] = 'Total'

In [19]:
new_table = new_table[['ONS Partner Geography', 'Period','Flow','Pink Book Services', 'Seasonal Adjustment', 'Measure Type','Value','Unit' ]]

In [21]:
if is_interactive():
    SubstancetinationFolder = Path('out')
    SubstancetinationFolder.mkdir(exist_ok=True, parents=True)
    new_table.to_csv(SubstancetinationFolder / ('tis.csv'), index = False)

In [22]:
new_table.head()

,ONS Partner Geography,Period,Flow,Pink Book Services,Seasonal Adjustment,Measure Type,Value,Unit
0,AT,quarter/2016-Q1,exports,Total,NSA,GBP Total,228,gbp-million
1,AT,quarter/2016-Q2,exports,Total,NSA,GBP Total,254,gbp-million
2,AT,quarter/2016-Q3,exports,Total,NSA,GBP Total,278,gbp-million
3,AT,quarter/2016-Q4,exports,Total,NSA,GBP Total,275,gbp-million
4,AT,quarter/2017-Q1,exports,Total,NSA,GBP Total,292,gbp-million
